In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
from latticeproteins.conformations import Conformations
from latticegpm.gpm import LatticeGenotypePhenotypeMap
from gpmap.graph import draw
from latticegpm.search import get_lowest_confs

from gpmap.evolve.models import fixation
from gpmap.graph.paths import paths_and_probabilities, path_probabilities
from gpmap.graph.draw.paths import flux
from gpmap.graph.draw.base import network

In [13]:
with open("scripts/sequences.txt", "r") as f:
    sequences = []
    for s in f.readlines():
        vals = s.strip().split(",")
        sequences.append((vals[0], vals[1][1:]))
        
c = Conformations(12, database_dir="database")

In [2]:
def edge_flux(G, source, target, width_scale=10, ax=None, figsize=(5,5), **settings):
    """
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    else:
        fig=None
        #fig = ax.get_figure()
    # Calculate fluxes on edges
    fluxes = flux(G, source, target)

    # Settings to the network
    defaults = dict(
        figsize=[2.5,2.5],
        n_height=9,
        n_width=9,
        n_linewidths=0,
        n_vmax=round(max(G.gpm.phenotypes),2),
        n_vmin=round(max(G.gpm.phenotypes),2),
        n_colorbar=True,
        e_arrows=False,
        l_alpha=0.0,
    )
    defaults.update(**settings)
    fig, ax, pos = network(G, e_width=width_scale*fluxes, ax=ax, **settings)
    return fig, ax

In [12]:
def draw_networks(G, G2, G3):
    """Draws three networks in one column."""

    fig = plt.figure(figsize=(12,9))
    gs1 = gridspec.GridSpec(3, 3)
    gs1.update(hspace=0)
    ax1 = plt.subplot(gs1[0, :])
    ax2 = plt.subplot(gs1[1, :])
    ax3 = plt.subplot(gs1[2, :])

    # FULL STATE
    s = dict(
        n_height=5,
        n_width=8,
        e_arrows=False,
        e_width=.2,
        e_edge_color="k",
        e_alpha=.5,
        l_font_size=6,
        l_alpha=0,
    )

    x = draw.network(G, ax=ax1, **s)

    # 2-STATE 
    s = dict(
        n_height=5,
        n_width=8,
        e_arrows=False,
        e_edge_color="k",
        e_alpha=.5,
        l_font_size=7,
        l_alpha=0,
    )

    x = edge_flux(G2, source, target, ax=ax2, **s)

    # 3-STATE
    s = dict(
        n_height=5,
        n_width=8,
        e_arrows=False,
        e_edge_color="k",
        e_alpha=.5,
        l_font_size=7,
        l_alpha=0,
    )

    x = edge_flux(G3, source, target, ax=ax3, **s)
    
    return fig

In [23]:
def make_network_figure(sequences, i):
    """"""
    wildtype = sequences[i][0]
    mutant = sequences[i][1]

    # Build spaces
    cs = get_lowest_confs(wildtype, 3, database="database")
    gpm = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c)
    gpm.phenotype_type = "fracfolded"
    G = gpm.add_networkx()
    G.add_evolutionary_model(fixation)

    gpm2 = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c)
    gpm2.set_partition_confs(cs[0:2])
    gpm2.phenotype_type = "fracfolded"
    G2 = gpm2.add_networkx()
    G2.add_evolutionary_model(fixation)

    gpm3 = LatticeGenotypePhenotypeMap.from_mutant(wildtype, mutant, c)
    gpm3.set_partition_confs(cs)
    gpm3.phenotype_type = "fracfolded"
    G3 = gpm3.add_networkx()
    G3.add_evolutionary_model(fixation)

    # Calculate paths
    mapping = gpm.map("genotypes", "indices")
    source = mapping[wildtype]
    target = mapping[mutant]
    paths, probs = paths_and_probabilities(G, source, target)

    mapping = gpm.map("genotypes", "indices")
    source = mapping[wildtype]
    target = mapping[mutant]
    paths2, probs2 = paths_and_probabilities(G2, source, target)

    mapping = gpm.map("genotypes", "indices")
    source = mapping[wildtype]
    target = mapping[mutant]
    paths3, probs3 = paths_and_probabilities(G3, source, target)

    fig = draw_networks(G, G2, G3)
    plt.close()
    fig.savefig("networks/network-" + str(i) + ".pdf", format="pdf")

In [26]:
for i in range(5,100):
    make_network_figure(sequences, i)

/home/zsailer/github/gpmap/gpmap/evolve/models.py:13: RuntimeWarning: overflow encountered in exp
  denominator = 1 - np.exp(-N * sij)
